# Setup
https://github.com/RobinMagnet/pyFM

```
conda activate
juptyer notebook
```

In [1]:
import numpy as np
from stl import mesh

In [2]:
from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# Load palpation pointcloud
If needed to convert to Mesh : https://github.com/OpenTopography/PointCloud_to_STL/blob/main/OT_VoxeltoSTL.ipynb

In [3]:
points = np.loadtxt("data/manual_palpation_1.csv", dtype=float, delimiter=',', 
                    skiprows=1, #Skip header
                    usecols=(7,8,9)) #Translations
# display(points)
print("Number of points in original file:",len(points))

mesh2=TriMesh(points, center=False)

Number of points in original file: 332


# Load model

## Convert STL to OFF

(Might not work depending on encoding...) From https://github.com/tforgione/model-converter-python :
python3 convert.py -i ../data/palpation_part1.stl -o ../data/palpation.off



In the meantime : https://imagetostl.com/convert/file/stl/to/off

## Load mesh

In [28]:
mesh1 = TriMesh('data/palpation_part1.off')#, area_normalize=True, center=False)
#mesh1 = TriMesh(mesh1.vertlist)
mesh2 = TriMesh('data/palpation.off')#, area_normalize=True, center=False)
#mesh2 = TriMesh(mesh2.vertlist)
#mesh2 = TriMesh(mesh2.vertlist[(mesh2.vertlist[:,1]>0.08) & (mesh2.vertlist[:,2]<0.),:])

#cond_idx, = np.where(mesh1.vertlist[:,1]>0)
#display(cond_idx)
#cond = np.isin(mesh1.facelist, cond_idx)
#cond = cond[:,0] & cond[:,1] & cond[:,2]
#mesh1 = TriMesh(mesh1.vertlist[mesh1.vertlist[:,1]>0,:], mesh1.facelist[cond,:])
#display(mesh1.facelist.shape)
display(np.amax(mesh1.vertlist, axis=0),np.amin(mesh1.vertlist, axis=0))
display(np.amax(mesh2.vertlist, axis=0),np.amin(mesh2.vertlist, axis=0))

array([0.12      , 0.023     , 0.03274316])

array([-5.551115e-17, -5.000000e-03, -2.400000e-02])

array([0.12      , 0.023     , 0.03274316])

array([-5.551115e-17, -1.000000e-05, -2.400000e-02])

# Compute functional map

**Display meshes data**

In [5]:
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')
double_plot(mesh1, mesh2)

Mesh 1 : 1648 vertices,  3292 faces
Mesh 2 : 8231 vertices, 16860 faces


/home/antoineh/.local/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


**Computing landmarks**

Results seems really dependent on landmarks (corresponding points indexs on the two meshes)

In [29]:
def getLandmarkIndex(points, coord):
    indexes = np.nonzero(np.isclose(points[:,0],coord[0]) & np.isclose(points[:,1],coord[1]) & np.isclose(points[:,2],coord[2]))
    return indexes[0][0] #Return first one found (throw error if none is found)

In [46]:
landmarks = np.array([
    #[np.argmax(mesh1.vertlist,axis=0)[0], np.argmax(mesh2.vertlist,axis=0)[0]],
    #[np.argmin(mesh1.vertlist,axis=0)[0], np.argmin(mesh2.vertlist,axis=0)[0]],
    #[np.argmax(mesh1.vertlist,axis=0)[1], np.argmax(mesh2.vertlist,axis=0)[1]],
    #[np.argmin(mesh1.vertlist,axis=0)[1], np.argmin(mesh2.vertlist,axis=0)[1]],
    #[np.argmax(mesh1.vertlist,axis=0)[2], np.argmax(mesh2.vertlist,axis=0)[2]],
    #[np.argmin(mesh1.vertlist,axis=0)[2], np.argmin(mesh2.vertlist,axis=0)[2]],
    #Pyramids
    [getLandmarkIndex(mesh1.vertlist, [0.09, 0.016, -0.012]), getLandmarkIndex(mesh2.vertlist, [0.09, 0.016, -0.012])],
    [getLandmarkIndex(mesh1.vertlist, [0.09, 0.016, 0.012]), getLandmarkIndex(mesh2.vertlist, [0.09, 0.016, 0.012])],
    [getLandmarkIndex(mesh1.vertlist, [0.11, 0.016, -0.012]), getLandmarkIndex(mesh2.vertlist, [0.11, 0.016, -0.012])],
    #Wall
    [getLandmarkIndex(mesh1.vertlist, [0., 0.02, -0.002]), getLandmarkIndex(mesh2.vertlist, [0., 0.02, -0.002])],
    [getLandmarkIndex(mesh1.vertlist, [0., 0.02, 0.002]), getLandmarkIndex(mesh2.vertlist, [0., 0.02, 0.002])],
    #Cylinder
    [getLandmarkIndex(mesh1.vertlist, [0.03, 0.017, 0.032743]), getLandmarkIndex(mesh2.vertlist, [0.03, 0.017, 0.032743])],
    [getLandmarkIndex(mesh1.vertlist, [0.01, 0.02, 0.022]), getLandmarkIndex(mesh2.vertlist, [0.01, 0.02, 0.022])],
])
display(landmarks)
#double_plot(mesh1, TriMesh(mesh1.vertlist[landmarks[:,0]]))

array([[214, 216],
       [104, 105],
       [432, 435],
       [641, 644],
       [643, 646],
       [828, 831],
       [687, 690]])

**Computing descriptors**

In [47]:
process_params = {
    'n_ev': (50,50),  # Number of eigenvalues on source and Target
    'n_descr': 100, #number of descriptors to consider
    'landmarks': landmarks, #np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 1,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 0.21 s
Computing 200 eigenvectors
	Done in 1.97 s

Computing descriptors
	Normalizing descriptors

	800 out of 800 possible descriptors kept


**Fitting the model**

In [48]:
fit_params = {
    'w_descr': 1e-1, #scaling for the descriptor preservation term
    'w_lap': 1e-3, #scaling of the laplacian commutativity term
    'w_dcomm': 1, #scaling of the multiplicative operator commutativity
    'w_orient': 0, #scaling of the orientation preservation term
    'orient_reversing':True, #Whether to use the orientation reversing term instead of the orientation preservation one
    'optinit':'zeros' #Initialization
}


model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 3.0e-13

Optimization :
	50 Ev on source - 50 Ev on Target
	Using 800 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e-01
		Descriptors commutativity :1.0e+00
		Laplacian commutativity :1.0e-03
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 30, nit : 27, warnflag : 0
	Done in 1.69 seconds


**Visualizing the associated point to point map**

In [52]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

/home/antoineh/.local/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## Refinining the Functioal Map

**ICP**

In [50]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/10 [00:00<?, ?it/s]

/home/antoineh/.local/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


**Zoomout**

In [51]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/15 [00:00<?, ?it/s]

(65, 65)


/home/antoineh/.local/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
